## Bayesian Interval Estimation 

In this example we compute a Bayesian credible interval using the RooStats `BayesianCalculator`.  
The interval is obtained from the Posterior probability function, which can be computed using the Bayes theorem

$$P( \mu | x ) = \frac { \int L (x | \mu , \nu ) \Pi ( \mu, \nu ) d\nu } { \int \int L (x | \mu , \nu ) \Pi ( \mu, \nu ) d \mu d\nu} $$

The class computes the posterior using adaptive numerical integration or Monte Carlo integration based on Vegas. 
This calculator works only for a single parameter of interest, but can work with few nuisance parameters.

In [ ]:
//%jsroot on

In [ ]:
//using namespace Roostats;
RooStats::SimpleInterval * interval = nullptr;
RooPlot * plot = nullptr;
RooAbsPdf * priorPdf = nullptr;

##### 1. Reading the model (Workspace) from input file

First part is just to access the workspace file and retrieve the model and the data 

In [ ]:
TString fileName = "CountingModelFixedB.root";
TString workspaceName = "w";
TString modelConfigName = "ModelConfig";
TString dataName = "data";
TString integrationType = "";  

In [ ]:
auto file = TFile::Open(fileName);

In [ ]:
auto w =  (RooWorkspace*) file->Get(workspaceName);
w->Print();
auto mc = (RooStats::ModelConfig*) w->obj(modelConfigName);
auto  data = w->data(dataName);

Print information about the number of events and background (works only in case of the specific counting model)

In [ ]:
std::cout << "number of observed events in the data : " << std::endl;
auto args = data->get(0);
if (args) args->Print("V");
else  Error("BayesianNumerical","Input data does not contain any event");
std::cout << "number of background events " << w->var("b")->getVal() << std::endl;

#### 2. Create and configure the `BayesianCalculator` class

In [ ]:
RooStats::BayesianCalculator bayesianCalc(*data,*mc);

##### Set a prior PDF, otherwise by default a uniform prior is used

In [ ]:
//priorPdf = new RooGenericPdf("priorPdf","priorPdf","1./sqrt(s)",RooArgList(*w->var("s")));
//priorPdf = new RooGenericPdf("priorPdf","priorPdf","1./s",RooArgList(*w->var("s")));
if (priorPdf) {
    bayesianCalc.SetPriorPdf(*priorPdf); 
    priorPdf->Print();
}

#### Set the interval Confidence Level

In [ ]:
bayesianCalc.SetConfidenceLevel(0.90);

##### Set the type of interval.

One can have : 

- shortest interval
- central interval (equal fraction of probability in the tails) 
- lower limit    
- upper limit  

The distinction between lower/upper and central interval is done by setting the left side tail fraction of probability.
The default here is central intervals

In [ ]:
//bayesianCalc.SetLeftSideTailFraction(0.5); // for central intervals
bayesianCalc.SetLeftSideTailFraction(0);  // for upper limit
//bayesianCalc.SetShortestInterval();       // for shortest intervals

Set integration type and configure it if needed. If nothing is specified use the default integrator in RooFit.

In [ ]:
RooAbsReal::defaultIntegratorConfig()->method1D().setLabel("RooAdaptiveGaussKronrodIntegrator1D");
//integrationType = "VEGAS";
//bayesianCalc.SetIntegrationType(integrationType); 
//bayesianCalc.SetNumIters(1000);

Compute the interval scanning the posterior function in the given number of points

In [ ]:
bayesianCalc.SetScanOfPosterior(100);

#### 3. Compute the interval

Run the calculator by computing the posterior probability using Bayes theorem. The nuisance parameters are integrated
(marginalized out). 

In [ ]:
interval = bayesianCalc.GetInterval();

In [ ]:
cout << 100*bayesianCalc.ConfidenceLevel() << "% interval is : ["<< interval->LowerLimit() << ", "<< interval->UpperLimit() <<"] "<<endl;

#### 4. Visualize the Interval

In [ ]:
plot = bayesianCalc.GetPosteriorPlot();

In [ ]:
plot->Draw(); gPad->Draw();

As continuation of the exercise one can varify the calculated interval for the simple Poisson case (with background=0) with the values shown in Table 1 of the paper of *R.D. Cousins, Why isn't every physicist a Bayesian, Am. J. Phys. 63, 398 (1995); http://dx.doi.org/10.1119/1.17901*